In [ ]:
import numpy as np
import os
import scipy.io as sio
import matplotlib.pyplot as plt
import torch

## EEG-UCI
Unzip the file first!

In [ ]:
import pandas as pd
import os
import scipy as sp
from tqdm import tqdm

In [ ]:
# EEG-UCI
base_folder = '../new_data/SMNI_CMI_TRAIN/'
r = os.walk(base_folder)
_, d, _ = list(r)[0]  

data_folder = 'co2a0000368'
f = list(os.walk(base_folder + str(d[0])))

L = 0
for directory in tqdm(d):
    f = list(os.walk(base_folder + str(directory)))
    #print(directory)
    for file in f[0][2]:
        if file[-2:] != 'gz':
            L = L + 1

In [ ]:
X = np.empty((L,1), dtype=object)
#X = []
i = 0
y = []
t = []
for directory in tqdm(d):
    f = list(os.walk(base_folder + str(directory)))
    #print(directory)
    for file in f[0][2]:
        if file[-2:] != 'gz':
            file_name = '../new_data/SMNI_CMI_TRAIN/' + directory + '/' + file
            #print(file_name)
            df = pd.read_csv(file_name, delimiter=' ', skiprows=4, names=['#', 'feat', 'seq', 'val'] )
            
            if directory[3] == 'a':
                lbl = 1
            else:
                lbl = 0
                
            y.append(lbl)
                
            # Features
            feats = np.unique(df['feat'].values)
            sam = []
            for chan in feats:
                sam.append(df[df['feat'] == chan]['val'].values[1:])

            sam = np.array(sam, dtype=float).T
            X[i] = [sam[::8]] 
            t.append(X[i][0].shape[0])
            #X.append([sam])
            i = i + 1

In [ ]:
# Prepare folds
arr = np.arange(len(np.array(y).T))
np.random.shuffle(arr)
partitions = np.array_split(arr, 5)
I = np.zeros(len(np.array(y).T),dtype='uint8')
for i in range(0,5):
    I[partitions[i]] = i+1


In [ ]:
dat = {}
dat['X'] = X
dat['y'] = y
dat['T'] = t

fol = {}
fol['folds'] = np.expand_dims(I,0)

In [ ]:
#sio.savemat("../new_data/ucieeg-new/ucieeg-new.mat", dat)
#sio.savemat("../new_data/ucieeg-new/ucieeg-new-5folds.mat", fol)

In [ ]:
dat = sio.loadmat("../new_data/ucieeg-new/ucieeg-new.mat")

In [ ]:
dat['X'][0][0].shape

## Physionet

In [ ]:
import pandas as pd
import os
import scipy as sp
from tqdm import tqdm
import time
import random
from sklearn.impute import SimpleImputer
np.random.seed(0)

out_name = 'physio-data'
base_folder = '../new_data/physionet-data/set-a/'
outcomes = pd.read_csv(base_folder[:-6]+ 'Outcomes-a.txt')

r = os.walk(base_folder)
_, _, f = list(r)[0]  
#L = len(f)


In [ ]:
pos_class = outcomes[outcomes['In-hospital_death'] == 1]['RecordID'].values
neg_class = outcomes[outcomes['In-hospital_death'] == 0]['RecordID'].values
sam_neg_class = random.sample(list(neg_class), len(pos_class))
sam_recs = list(pos_class) + sam_neg_class
random.shuffle(sam_recs)
L = len(sam_recs)
sam_files = [str(rec) + '.txt' for rec in sam_recs]

In [ ]:
feats = ['Albumin', 
    'ALP', 
    'ALT', 
    'AST',
    'Bilirubin',
    'BUN', 
    'Cholesterol',
    'Creatinine',
    'DiasABP',
    'FiO2',
    'GCS', 
    'Glucose', 
    'HCO3', 
    'HCT', 
    'HR', 
    'K', 
    'Lactate', 
    'Mg', 
    'MAP', 
    'MechVent', 
    'Na', 
    'NIDiasABP',
    'NIMAP',
    'NISysABP', 
	'PaCO2',
    'PaO2', 
    'pH', 
    'Platelets',
    'RespRate',
    'SaO2', 
    'SysABP', 
    'Temp',    
    'TropI',
    'TropT',
    'Urine',
    'WBC',
    'Weight']
len(feats)

In [ ]:

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
def hr_conv(a):
    return int(a[0:2])

X = np.empty((L,1), dtype=object)
#X = []
i = 0
y = []
t = []
seen_feats = []
#for directory in tqdm(d):
#    f = list(os.walk(base_folder + str(directory)))
    #print(directory)
track_max = []
for file in sam_files:
    if file[-3:] == 'txt':
        file_name = base_folder + file
            #print(file_name)
        df = pd.read_csv(file_name, delimiter=',', skiprows=6, names=['time', 'feat', 'val'])
        df['hour'] = list(map(hr_conv, df['time'].values))
        track_max.append(max(df['hour']))
        if max(df['hour']) == 49:
            
            break
                 
        y.append(outcomes[outcomes['RecordID'] == int(file[:-4])]['In-hospital_death'].values[0])
        
      
        # Features
        #feats = np.unique(df['feat'].values)
        seen_feats.append(list(np.unique(df['feat'].values)))
        sam = []
        for chan in feats:
            
            if df[df['feat'] == chan].empty:
                sen_dat = np.zeros([49,1])
                sam.append(sen_dat)
            else:
                sen_dat = np.empty([49,1])
                sen_dat[:] = np.nan
                sen_dat[list(df[df['feat'] == chan].groupby('hour').groups.keys())] = df[df['feat'] == chan].groupby('hour').mean().values

            
                sam.append(imp.fit_transform(sen_dat))
        
        sam = np.squeeze(np.array(sam, dtype=float).T)
        X[i] = [sam] 
        t.append(sam.shape[0])
         #X.append([sam])
        i = i + 1

                        

In [ ]:
# Prepare folds
arr = np.arange(len(np.array(y).T))
np.random.shuffle(arr)
partitions = np.array_split(arr, 5)
I = np.zeros(len(np.array(y).T),dtype='uint8')
for i in range(0,5):
    I[partitions[i]] = i+1

dat = {}
dat['X'] = X
dat['y'] = y
dat['T'] = t

fol = {}

fol['folds'] = np.expand_dims(I,0)   

In [ ]:
#sio.savemat('../new_data/' + out_name + '/' + out_name + '.mat', dat)
#sio.savemat('../new_data/' + out_name + '/' + out_name + '-5folds.mat', fol)

## RAS data

In [ ]:
import numpy as np
import os
import scipy.io as sio
import matplotlib.pyplot as plt
import torch
from argparse import ArgumentParser
import pickle
np.random.seed(0)



os.chdir('../new_data/RAS/')
from utils import process_entry
import dataloaders
#parser = ArgumentParser()
#parser.add_argument('--input_kinema28062466tics_folder', type=str, default='../new_data/RAS')
#args = parser.parse_args()

#input_kinematics_folder = args.input_kinematics_folder
input_kinematics_folder ='../../new_data/RAS/kinematics/'
input_videos_folder = '../multi_modal/processed_data/videos/' # <path_to_videos>

tasks_names = ['AB_label_needle_positionB', 'BC_label_needle_entry_angleC', 'CD_label_needle_driving_1D', 'FG_label_needle_driving_2FG']

out_name = 'ras-data-bcea'
task_names = [tasks_names[1] ]
task_abbrv = ''

i = 0

for task_name in task_names:
    task_file = task_name + str('.p')
   
    data_f = pickle.load( open( input_kinematics_folder+'{}'.format(task_file), "rb" ) )#.format(args.task_file), "rb" ) )
    
    if task_abbrv != '' :
        data_p = {k + '_' + task_abbrv[i]:(process_entry(v1),v2) for k,(v1,v2) in data_f.items()}
    else:
        data_p = {k:(process_entry(v1),v2) for k,(v1,v2) in data_f.items()}
           
    data_loader = dataloaders.SpatialDataloader(batch_size=32,
                                            num_workers=8,
                                            data_path=os.path.join(input_videos_folder, task_name),
                                            #seed=args.seed)
                                            seed=42)
    
    _, _, test_video_p = data_loader.run()

    
    if task_abbrv != '' :
        test_video_p = {k + '_' + task_abbrv[i]:int(v) for k,v in test_video_p.items()}
    
    
    
    if i == 0:
        data = dict(data_p)
        test_video = dict(test_video_p)
        
    else:
        data.update(data_p)
        test_video.update(test_video_p)
    

        
    i = i + 1
    
sequence_lengths = [data[row][0].shape[0] for row in data]
trunc_len = int(np.percentile(sequence_lengths, 95))
V, X, y = [], [], []

for row in data:
    V.append(row)
    #X.append(torch.from_numpy(data[row][0][-trunc_len:]).float())   
    X.append(torch.from_numpy(data[row][0]).float()) 
    y.append(data[row][1])


In [ ]:
#[x.shape[0] for x in X]
int(np.percentile(sequence_lengths, 50))
plt.hist(sequence_lengths, 150)
plt.show()

In [ ]:

seq_len = [X[i].shape[0] for i in range(0, len(X))]
low = 2
high = np.max(seq_len)
L = np.sum([(X[i].shape[0] > low) & (X[i].shape[0] <= high) for i in range(0, len(X))])
Xx = np.empty((L,1), dtype=object)
#X = []
i = 0
yy = []
t = []
ii = 0
d = 0
for x in X:
    sam = np.array(x.cpu().numpy(), dtype=float)
    #sam = np.array(x, dtype=float).T
    
    if (sam.shape[0] > low) & (sam.shape[0] <= high):
       # print(sam.shape)
        Xx[ii] = [sam] 
        yy.append(int(y[i]))
        t.append(sam.shape[0])
         #X.append([sam])
        ii = ii + 1
    else:
        d = d + 1
    i = i + 1
    

In [ ]:
L

In [ ]:
# Prepare folds
arr = np.arange(len(np.array(yy).T))
np.random.shuffle(arr)
partitions = np.array_split(arr, 5)
I = np.zeros(len(np.array(yy).T),dtype='uint8')
for i in range(0,5):
    I[partitions[i]] = i+1


In [ ]:
dat = {}
dat['X'] = Xx
dat['y'] = yy
dat['T'] = t

fol = {}
fol['folds'] = np.expand_dims(I,0)

In [ ]:

#sio.savemat('../new_data/' + out_name + '/' + out_name + '.mat', dat)
#sio.savemat('../new_data/' + out_name + '/' + out_name + '-5folds.mat', fol)

In [ ]:
dat = sio.loadmat("../new_data/ras-data/ras-data.mat")

In [ ]:
plt.plot(X[2])
plt.show()

## Asphalt Regularity Dataset

In [ ]:
import pandas as pd
import os
import scipy as sp
from tqdm import tqdm
import time
import random
from sklearn.impute import SimpleImputer
import h5py
from scipy.io import arff
np.random.seed(0)

out_name = 'ucr-data-aspreg'
base_folder = '../sdtw_data' # <path_to_data>
 
r = os.walk(base_folder)
_, f, _ = list(r)[0]  

In [ ]:
file_name_tr = base_folder+ '/' + 'AsphaltRegularity' + '/' + 'AsphaltRegularity' + '_TRAIN.arff'
file_name_te = base_folder+ '/' + 'AsphaltRegularity' + '/' + 'AsphaltRegularity' + '_TEST.arff'
dat_arff_tr = arff.loadarff(file_name_tr)
dat_arff_te = arff.loadarff(file_name_te)

In [ ]:
dat_tr = pd.DataFrame(dat_arff_tr[0])
dat_te = pd.DataFrame(dat_arff_te[0])

In [ ]:
dat_tr.target = pd.factorize(dat_tr.target)[0]
dat_te.target = pd.factorize(dat_te.target)[0]

In [ ]:
data = pd.concat([dat_tr, dat_te], axis=0)
data = data.reset_index(drop=True)
labels = list(data['target'])

In [ ]:
data = data.drop(['target'], axis=1)
data.columns

In [ ]:
L = data.shape[0]
X = np.empty((L,1), dtype=object)
#X = []
i = 0
y = []
t = []

for i in range(0, data.shape[0]):
    y.append(labels[i])
    sam = np.squeeze(np.array(list(data.iloc[i][data.iloc[i].notnull()].T)))
    sam = np.expand_dims(sam, 1)
    X[i] = [sam] 
    t.append(sam.shape[0])


In [ ]:
# Prepare folds
arr = np.arange(len(np.array(y).T))
np.random.shuffle(arr)
partitions = np.array_split(arr, 5)
I = np.zeros(len(np.array(y).T),dtype='uint8')
for i in range(0,5):
    I[partitions[i]] = i+1

dat = {}
dat['X'] = X
dat['y'] = y
dat['T'] = t

fol = {}

fol['folds'] = np.expand_dims(I,0)  

In [ ]:
#sio.savemat('../new_data/' + out_name + '/' + out_name + '.mat', dat)
#sio.savemat('../new_data/' + out_name + '/' + out_name + '-5folds.mat', fol)